<a href="https://colab.research.google.com/github/ashishpatel26/Rapidsai_Machine_learning_on_GPU/blob/main/10_Min_to_Pandas_and_Cudf_Comparision_on_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://github.com/ashishpatel26/Rapidsai_Machine_learning_on_GPU/raw/main/images/rapidsailogo.jpg?raw=true)

🤩**About Rapids**🤩

- The RAPIDS suite of open source software libraries aim to enable execution of end-to-end data science and analytics pipelines entirely on GPUs. It relies on NVIDIA® CUDA® primitives for low-level compute optimization, but exposing that GPU parallelism and high-bandwidth memory speed through user-friendly Python interfaces.
![](https://github.com/rapidsai/cudf/raw/branch-21.08/img/rapids_arrow.png)

# 10 Min to Pandas and cuDF Comparision on GPU

🎯 Built based on the Apache Arrow columnar memory format, cuDF is a GPU DataFrame library for loading, joining, aggregating, filtering, and otherwise manipulating data.

🎯 cuDF provides a pandas-like API that will be familiar to data engineers & data scientists, so they can use it to easily accelerate their workflows without going into the details of CUDA programming.

### **1.Installation of cuDF**

> 🏹**Conda installation** - 📥Recommended

```console
# for CUDA 11.0
conda install -c rapidsai -c nvidia -c numba -c conda-forge \
    cudf=21.06 python=3.7 cudatoolkit=11.0

# or, for CUDA 11.2
conda install -c rapidsai -c nvidia -c numba -c conda-forge \
    cudf=21.06 python=3.7 cudatoolkit=11.2
```
> 🏹**For Nightly Version**

```console
# for CUDA 11.0
conda install -c rapidsai-nightly -c nvidia -c numba -c conda-forge \
    cudf python=3.7 cudatoolkit=11.0

# or, for CUDA 11.2
conda install -c rapidsai-nightly -c nvidia -c numba -c conda-forge \
    cudf python=3.7 cudatoolkit=11.2
```
> 🏹**PIP Installation**

```console
!pip install cudf
```

> 🏹**Colab Installation** - [![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1rY7Ln6rEE1pOlfSHCYOVaqt8OvDO35J0#forceEdit=true&offline=true&sandboxMode=true
)

In [ ]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.7/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.7/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

### **2.Import Libraries**

In [5]:
import os
import cupy as cp
import pandas as pd
import cudf
import dask_cudf
import time

cp.random.seed(2021)

In [4]:
print(cudf.__version__)
print(dask_cudf.__version__)
print(pd.__version__)

0.19.2
0.19.2
1.1.5


### **3.Object Creation**

| Framework  | Pandas | cudf | daskcudf |
| ---- | ---- | ---- | ---- |
| Time | 3.84ms| 9.6ms| 18.5ms |

In [10]:
%%time
# 1. Pandas
s = pd.Series([1,2,3,None, 4])
print(s)

0    1.0
1    2.0
2    3.0
3    NaN
4    4.0
dtype: float64
CPU times: user 2.35 ms, sys: 0 ns, total: 2.35 ms
Wall time: 3.84 ms


In [11]:
%%time
# 2. cudf
s = cudf.Series([1,2,3,None, 4])
print(s)

0       1
1       2
2       3
3    <NA>
4       4
dtype: int64
CPU times: user 5.37 ms, sys: 35 µs, total: 5.41 ms
Wall time: 9.6 ms


In [12]:
%%time
ds = dask_cudf.from_cudf(s, npartitions=2)
print(ds.compute())

0       1
1       2
2       3
3    <NA>
4       4
dtype: int64
CPU times: user 8.22 ms, sys: 1.77 ms, total: 10 ms
Wall time: 18.5 ms


### **4.Loading Dataset**

| Framework  | Pandas | cudf | daskcudf |
| ---- | ---- | ---- | ---- |
| Time | 2.42s| 1.52s| 1.62s |


In [16]:
%%time
# 1.Pandas
data = pd.read_csv("https://storage.googleapis.com/industryanalytics/LoanDefaultData.csv")
display(data.head(2))

,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
0,180675,2007,Andhra Pradesh,01/12/2007,1032009,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,Low,C,73000,25000,10.91,36 months,22.13,13650.38,8767.32,2207.65,817.41,1
1,85781,2007,Rajasthan,01/06/2007,1072010,0.5,RENT,Low,INDIVIDUAL,other,Low,C,40000,1400,10.91,36 months,8.61,1663.04,1400.00,0.00,45.78,0


CPU times: user 1.69 s, sys: 171 ms, total: 1.86 s
Wall time: 2.42 s


In [17]:
%%time
# 2.cudf
data_cudf = cudf.read_csv("https://storage.googleapis.com/industryanalytics/LoanDefaultData.csv")
display(data_cudf.head(2))

,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
0,180675,2007,Andhra Pradesh,01/12/2007,1032009,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,Low,C,73000,25000,10.91,36 months,22.13,13650.38,8767.32,2207.65,817.41,1
1,85781,2007,Rajasthan,01/06/2007,1072010,0.5,RENT,Low,INDIVIDUAL,other,Low,C,40000,1400,10.91,36 months,8.61,1663.04,1400.00,0.00,45.78,0


CPU times: user 587 ms, sys: 314 ms, total: 901 ms
Wall time: 1.52 s


In [20]:
%%time
# 3.dask_cudf
data_daskcudf = dask_cudf.from_cudf(cudf.read_csv("https://storage.googleapis.com/industryanalytics/LoanDefaultData.csv"), npartitions=8)
display(data_daskcudf.head(2))

,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
0,180675,2007,Andhra Pradesh,01/12/2007,1032009,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,Low,C,73000,25000,10.91,36 months,22.13,13650.38,8767.32,2207.65,817.41,1
1,85781,2007,Rajasthan,01/06/2007,1072010,0.5,RENT,Low,INDIVIDUAL,other,Low,C,40000,1400,10.91,36 months,8.61,1663.04,1400.00,0.00,45.78,0


CPU times: user 622 ms, sys: 305 ms, total: 927 ms
Wall time: 1.62 s


### **5.Convert Pandas data to cudf and dask_cudf**

| Framework  | Pandas to cudf | Pandas to daskcudf |
| ---- | ---- | ---- | 
| Time | 2.73s| 2.79s| 

In [21]:
%%time
# 1. cudf from pandas
data = pd.read_csv("https://storage.googleapis.com/industryanalytics/LoanDefaultData.csv")
cudf_data = cudf.DataFrame.from_pandas(data)
cudf_data

CPU times: user 2.19 s, sys: 150 ms, total: 2.34 s
Wall time: 2.73 s


In [22]:
%%time
# 2. dask_cudf from pandas
data = pd.read_csv("https://storage.googleapis.com/industryanalytics/LoanDefaultData.csv")
cudf_data = cudf.DataFrame.from_pandas(data)
daskcudf_data = dask_cudf.from_cudf(cudf_data, npartitions=2)

CPU times: user 2.11 s, sys: 252 ms, total: 2.36 s
Wall time: 2.79 s


### **6.Viewing Data**

| Framework  | Pandas | cudf | daskcudf |
| ---- | ---- | ---- | ---- |
| Time | 241 µs| 4.32ms| 12.7ms |

In [23]:
%%time
# 1.Pandas
data.head()

CPU times: user 232 µs, sys: 2 µs, total: 234 µs
Wall time: 241 µs


,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
0,180675,2007,Andhra Pradesh,01/12/2007,1032009,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,Low,C,73000,25000,10.91,36 months,22.13,13650.38,8767.32,2207.65,817.41,1
1,85781,2007,Rajasthan,01/06/2007,1072010,0.5,RENT,Low,INDIVIDUAL,other,Low,C,40000,1400,10.91,36 months,8.61,1663.04,1400.00,0.00,45.78,0
2,85675,2007,Manipur,01/06/2007,1062010,10.0,RENT,Low,INDIVIDUAL,other,High,E,25000,1000,14.07,36 months,16.27,1231.38,1000.00,0.00,34.21,0
3,84918,2007,Andhra Pradesh,01/09/2007,1042008,10.0,MORTGAGE,Low,INDIVIDUAL,other,Low,A,65000,5000,7.43,36 months,0.28,5200.44,5000.00,0.00,155.38,0
4,84670,2007,Arunachal Pradesh,01/06/2007,1082009,10.0,MORTGAGE,High,INDIVIDUAL,other,Low,A,300000,5000,7.75,36 months,5.38,5565.65,5000.00,0.00,156.11,0


In [24]:
%%time
# 2.cudf
cudf_data.head()

CPU times: user 4.55 ms, sys: 0 ns, total: 4.55 ms
Wall time: 4.32 ms


,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
0,180675,2007,Andhra Pradesh,01/12/2007,1032009,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,Low,C,73000,25000,10.91,36 months,22.13,13650.38,8767.32,2207.65,817.41,1
1,85781,2007,Rajasthan,01/06/2007,1072010,0.5,RENT,Low,INDIVIDUAL,other,Low,C,40000,1400,10.91,36 months,8.61,1663.04,1400.00,0.00,45.78,0
2,85675,2007,Manipur,01/06/2007,1062010,10.0,RENT,Low,INDIVIDUAL,other,High,E,25000,1000,14.07,36 months,16.27,1231.38,1000.00,0.00,34.21,0
3,84918,2007,Andhra Pradesh,01/09/2007,1042008,10.0,MORTGAGE,Low,INDIVIDUAL,other,Low,A,65000,5000,7.43,36 months,0.28,5200.44,5000.00,0.00,155.38,0
4,84670,2007,Arunachal Pradesh,01/06/2007,1082009,10.0,MORTGAGE,High,INDIVIDUAL,other,Low,A,300000,5000,7.75,36 months,5.38,5565.65,5000.00,0.00,156.11,0


In [28]:
%%time
# 3.daskcudf
daskcudf_data.head()

CPU times: user 9.35 ms, sys: 0 ns, total: 9.35 ms
Wall time: 12.7 ms


,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
0,180675,2007,Andhra Pradesh,01/12/2007,1032009,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,Low,C,73000,25000,10.91,36 months,22.13,13650.38,8767.32,2207.65,817.41,1
1,85781,2007,Rajasthan,01/06/2007,1072010,0.5,RENT,Low,INDIVIDUAL,other,Low,C,40000,1400,10.91,36 months,8.61,1663.04,1400.00,0.00,45.78,0
2,85675,2007,Manipur,01/06/2007,1062010,10.0,RENT,Low,INDIVIDUAL,other,High,E,25000,1000,14.07,36 months,16.27,1231.38,1000.00,0.00,34.21,0
3,84918,2007,Andhra Pradesh,01/09/2007,1042008,10.0,MORTGAGE,Low,INDIVIDUAL,other,Low,A,65000,5000,7.43,36 months,0.28,5200.44,5000.00,0.00,155.38,0
4,84670,2007,Arunachal Pradesh,01/06/2007,1082009,10.0,MORTGAGE,High,INDIVIDUAL,other,Low,A,300000,5000,7.75,36 months,5.38,5565.65,5000.00,0.00,156.11,0


### **7.Sorting Values**

| Framework  | Pandas | cudf | daskcudf |
| ---- | ---- | ---- | ---- |
| Time | 149ms| 37.8ms| 4.67s |

In [29]:
%%time
# 1.Pandas
data.sort_values('cust_id').head()

CPU times: user 143 ms, sys: 0 ns, total: 143 ms
Wall time: 149 ms


,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
7853,54734,2009,Haryana,01/08/2009,1102011,0.5,RENT,Low,INDIVIDUAL,debt_consolidation,Low,B,85000,25000,11.89,36 months,19.48,29324.32,25000.00,0.00,829.10,0
614,55521,2008,Karnataka,01/07/2008,1032010,0.5,RENT,Low,INDIVIDUAL,debt_consolidation,High,F,30000,1000,16.08,36 months,23.84,1207.76,999.99,0.00,35.20,0
615,55742,2008,West Bengal,01/05/2008,1062011,0.5,RENT,Low,INDIVIDUAL,credit_card,Low,B,65000,7000,10.71,36 months,14.29,8215.45,7000.00,0.00,228.22,0
636,56413,2008,Nagaland,01/04/2008,1102008,10.0,MORTGAGE,Medium,INDIVIDUAL,debt_consolidation,High,F,189500,7000,16.08,36 months,22.47,1231.90,783.46,0.25,246.38,1
470508,56705,2015,Andhra Pradesh,01/11/2015,1012016,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,Low,B,33500,11000,9.99,36 months,18.38,376.25,263.31,0.00,354.89,0


In [30]:
%%time
# 2.cudf
cudf_data.sort_values('cust_id').head()

CPU times: user 33.1 ms, sys: 3.84 ms, total: 36.9 ms
Wall time: 37.8 ms


,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
7853,54734,2009,Haryana,01/08/2009,1102011,0.5,RENT,Low,INDIVIDUAL,debt_consolidation,Low,B,85000,25000,11.89,36 months,19.48,29324.32,25000.00,0.00,829.10,0
614,55521,2008,Karnataka,01/07/2008,1032010,0.5,RENT,Low,INDIVIDUAL,debt_consolidation,High,F,30000,1000,16.08,36 months,23.84,1207.76,999.99,0.00,35.20,0
615,55742,2008,West Bengal,01/05/2008,1062011,0.5,RENT,Low,INDIVIDUAL,credit_card,Low,B,65000,7000,10.71,36 months,14.29,8215.45,7000.00,0.00,228.22,0
636,56413,2008,Nagaland,01/04/2008,1102008,10.0,MORTGAGE,Medium,INDIVIDUAL,debt_consolidation,High,F,189500,7000,16.08,36 months,22.47,1231.90,783.46,0.25,246.38,1
470508,56705,2015,Andhra Pradesh,01/11/2015,1012016,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,Low,B,33500,11000,9.99,36 months,18.38,376.25,263.31,0.00,354.89,0


In [31]:
%%time
# 3.daskcudf
daskcudf_data.sort_values('cust_id').head()

CPU times: user 3.75 s, sys: 211 ms, total: 3.96 s
Wall time: 4.67 s


,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
7853,54734,2009,Haryana,01/08/2009,1102011,0.5,RENT,Low,INDIVIDUAL,debt_consolidation,Low,B,85000,25000,11.89,36 months,19.48,29324.32,25000.00,0.00,829.10,0
614,55521,2008,Karnataka,01/07/2008,1032010,0.5,RENT,Low,INDIVIDUAL,debt_consolidation,High,F,30000,1000,16.08,36 months,23.84,1207.76,999.99,0.00,35.20,0
615,55742,2008,West Bengal,01/05/2008,1062011,0.5,RENT,Low,INDIVIDUAL,credit_card,Low,B,65000,7000,10.71,36 months,14.29,8215.45,7000.00,0.00,228.22,0
636,56413,2008,Nagaland,01/04/2008,1102008,10.0,MORTGAGE,Medium,INDIVIDUAL,debt_consolidation,High,F,189500,7000,16.08,36 months,22.47,1231.90,783.46,0.25,246.38,1
470508,56705,2015,Andhra Pradesh,01/11/2015,1012016,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,Low,B,33500,11000,9.99,36 months,18.38,376.25,263.31,0.00,354.89,0


### **8.Selection**

| Framework  | Pandas | cudf | daskcudf |
| ---- | ---- | ---- | ---- |
| Time | 46.3µs| 1.18ms| 7.37ms |

In [32]:
%%time
# 1. Pandas
data["state"]

CPU times: user 42 µs, sys: 0 ns, total: 42 µs
Wall time: 46.3 µs


0            Andhra Pradesh
1                 Rajasthan
2                   Manipur
3            Andhra Pradesh
4         Arunachal Pradesh
                ...        
887374             Nagaland
887375            Telangana
887376            Meghalaya
887377                Bihar
887378               Odisha
Name: state, Length: 887379, dtype: object

In [33]:
%%time
# 2. cudf
cudf_data["state"]

CPU times: user 1.17 ms, sys: 26 µs, total: 1.19 ms
Wall time: 1.18 ms


0            Andhra Pradesh
1                 Rajasthan
2                   Manipur
3            Andhra Pradesh
4         Arunachal Pradesh
                ...        
887374             Nagaland
887375            Telangana
887376            Meghalaya
887377                Bihar
887378               Odisha
Name: state, Length: 887379, dtype: object

In [35]:
%%time
# 3. cudf
daskcudf_data["state"].compute()

CPU times: user 5.27 ms, sys: 873 µs, total: 6.15 ms
Wall time: 7.37 ms


0            Andhra Pradesh
1                 Rajasthan
2                   Manipur
3            Andhra Pradesh
4         Arunachal Pradesh
                ...        
887374             Nagaland
887375            Telangana
887376            Meghalaya
887377                Bihar
887378               Odisha
Name: state, Length: 887379, dtype: object

### **9.Selection By label**

| Framework  | Pandas | cudf | daskcudf |
| ---- | ---- | ---- | ---- |
| Time | 1.4ms| 1.82ms| 7.45ms |

In [41]:
%%time
# 1.Pandas
data.loc[1:100, ["state","income_type"]]

CPU times: user 1.38 ms, sys: 61 µs, total: 1.44 ms
Wall time: 1.4 ms


,state,income_type
1,Rajasthan,Low
2,Manipur,Low
3,Andhra Pradesh,Low
4,Arunachal Pradesh,High
5,Chhattisgarh,Low
...,...,...
96,Odisha,Low
97,Kerala,Low
98,Madhya Pradesh,Low
99,Arunachal Pradesh,Medium


In [51]:
%%time
# 2.cudf
cudf_data.loc[1:100, ["state","income_type"]]

CPU times: user 1.81 ms, sys: 0 ns, total: 1.81 ms
Wall time: 1.82 ms


,state,income_type
1,Rajasthan,Low
2,Manipur,Low
3,Andhra Pradesh,Low
4,Arunachal Pradesh,High
5,Chhattisgarh,Low
...,...,...
96,Odisha,Low
97,Kerala,Low
98,Madhya Pradesh,Low
99,Arunachal Pradesh,Medium


In [52]:
%%time
# 3.daskcudf
daskcudf_data.loc[1:100, ["state","income_type"]].compute()

CPU times: user 7.13 ms, sys: 0 ns, total: 7.13 ms
Wall time: 7.45 ms


,state,income_type
1,Rajasthan,Low
2,Manipur,Low
3,Andhra Pradesh,Low
4,Arunachal Pradesh,High
5,Chhattisgarh,Low
...,...,...
96,Odisha,Low
97,Kerala,Low
98,Madhya Pradesh,Low
99,Arunachal Pradesh,Medium


### **10.Selection by Position**

| Framework  | Pandas | cudf | daskcudf |
| ---- | ---- | ---- | ---- |
| Time | 1.4ms| 1.82ms| 30.1ms |

In [56]:
%%time
# 1.Pandas
data.iloc[4:10, 0:2].head()

CPU times: user 1.41 ms, sys: 66 µs, total: 1.48 ms
Wall time: 1.99 ms


,cust_id,year
4,84670,2007
5,84098,2007
6,83979,2007
7,85818,2007
8,83489,2007


In [57]:
%%time
# 2.cudf
cudf_data.iloc[4:10, 0:2].head()

CPU times: user 2.87 ms, sys: 0 ns, total: 2.87 ms
Wall time: 2.84 ms


,cust_id,year
4,84670,2007
5,84098,2007
6,83979,2007
7,85818,2007
8,83489,2007


In [61]:
%%time
# 3.daskcudf
daskcudf_data.compute().iloc[4:10, 0:2].head()

CPU times: user 14 ms, sys: 14.9 ms, total: 28.9 ms
Wall time: 30.1 ms


,cust_id,year
4,84670,2007
5,84098,2007
6,83979,2007
7,85818,2007
8,83489,2007


### **11.Boolean Indexing**

| Framework  | Pandas | cudf | daskcudf |
| ---- | ---- | ---- | ---- |
| Time | 182ms| 34.7ms| 9.8ms |

In [67]:
%%time
# 1. Pandas
data[data.cust_id>83000]

CPU times: user 68.8 ms, sys: 110 ms, total: 179 ms
Wall time: 182 ms


,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
0,180675,2007,Andhra Pradesh,01/12/2007,1032009,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,Low,C,73000,25000,10.91,36 months,22.13,13650.38,8767.32,2207.65,817.41,1
1,85781,2007,Rajasthan,01/06/2007,1072010,0.5,RENT,Low,INDIVIDUAL,other,Low,C,40000,1400,10.91,36 months,8.61,1663.04,1400.00,0.00,45.78,0
2,85675,2007,Manipur,01/06/2007,1062010,10.0,RENT,Low,INDIVIDUAL,other,High,E,25000,1000,14.07,36 months,16.27,1231.38,1000.00,0.00,34.21,0
3,84918,2007,Andhra Pradesh,01/09/2007,1042008,10.0,MORTGAGE,Low,INDIVIDUAL,other,Low,A,65000,5000,7.43,36 months,0.28,5200.44,5000.00,0.00,155.38,0
4,84670,2007,Arunachal Pradesh,01/06/2007,1082009,10.0,MORTGAGE,High,INDIVIDUAL,other,Low,A,300000,5000,7.75,36 months,5.38,5565.65,5000.00,0.00,156.11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887374,68616825,2015,Nagaland,01/12/2015,1012016,2.0,RENT,Medium,INDIVIDUAL,debt_consolidation,Low,B,130000,20000,9.17,36 months,12.77,0.00,0.00,0.00,637.58,0
887375,68616851,2015,Telangana,01/12/2015,1012016,10.0,RENT,Low,INDIVIDUAL,credit_card,Low,A,52000,12500,7.49,36 months,6.83,0.00,0.00,0.00,388.78,0
887376,68616867,2015,Meghalaya,01/12/2015,1012016,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,High,C,80000,16000,14.85,60 months,19.59,247.39,181.39,0.00,379.39,0
887377,68616501,2015,Bihar,01/12/2015,1012016,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,High,D,100000,25000,18.49,60 months,5.58,0.00,0.00,0.00,641.52,0


In [68]:
%%time
# 2. cudf
cudf_data[cudf_data.cust_id>83000]

CPU times: user 24.5 ms, sys: 6.14 ms, total: 30.7 ms
Wall time: 34.7 ms


,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
0,180675,2007,Andhra Pradesh,01/12/2007,1032009,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,Low,C,73000,25000,10.91,36 months,22.13,13650.38,8767.32,2207.65,817.41,1
1,85781,2007,Rajasthan,01/06/2007,1072010,0.5,RENT,Low,INDIVIDUAL,other,Low,C,40000,1400,10.91,36 months,8.61,1663.04,1400.00,0.00,45.78,0
2,85675,2007,Manipur,01/06/2007,1062010,10.0,RENT,Low,INDIVIDUAL,other,High,E,25000,1000,14.07,36 months,16.27,1231.38,1000.00,0.00,34.21,0
3,84918,2007,Andhra Pradesh,01/09/2007,1042008,10.0,MORTGAGE,Low,INDIVIDUAL,other,Low,A,65000,5000,7.43,36 months,0.28,5200.44,5000.00,0.00,155.38,0
4,84670,2007,Arunachal Pradesh,01/06/2007,1082009,10.0,MORTGAGE,High,INDIVIDUAL,other,Low,A,300000,5000,7.75,36 months,5.38,5565.65,5000.00,0.00,156.11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887374,68616825,2015,Nagaland,01/12/2015,1012016,2.0,RENT,Medium,INDIVIDUAL,debt_consolidation,Low,B,130000,20000,9.17,36 months,12.77,0.00,0.00,0.00,637.58,0
887375,68616851,2015,Telangana,01/12/2015,1012016,10.0,RENT,Low,INDIVIDUAL,credit_card,Low,A,52000,12500,7.49,36 months,6.83,0.00,0.00,0.00,388.78,0
887376,68616867,2015,Meghalaya,01/12/2015,1012016,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,High,C,80000,16000,14.85,60 months,19.59,247.39,181.39,0.00,379.39,0
887377,68616501,2015,Bihar,01/12/2015,1012016,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,High,D,100000,25000,18.49,60 months,5.58,0.00,0.00,0.00,641.52,0


In [69]:
%%time
# 3. daskcudf
daskcudf_data[daskcudf_data.cust_id>83000]

CPU times: user 3.59 ms, sys: 0 ns, total: 3.59 ms
Wall time: 9.8 ms


,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
npartitions=2,,,,,,,,,,,,,,,,,,,,,,
0,int64,int64,object,object,int64,float64,object,object,object,object,object,object,int64,int64,float64,object,float64,float64,float64,float64,float64,int64
443690,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887378,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


### **12.Query API**

| Framework  | Pandas | cudf | daskcudf |
| ---- | ---- | ---- | ---- |
| Time | 189ms| 178ms| 75.5ms |


In [74]:
%%time
# 1. pandas
data.query("cust_id > 83000")

CPU times: user 90.2 ms, sys: 86.3 ms, total: 177 ms
Wall time: 189 ms


,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
0,180675,2007,Andhra Pradesh,01/12/2007,1032009,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,Low,C,73000,25000,10.91,36 months,22.13,13650.38,8767.32,2207.65,817.41,1
1,85781,2007,Rajasthan,01/06/2007,1072010,0.5,RENT,Low,INDIVIDUAL,other,Low,C,40000,1400,10.91,36 months,8.61,1663.04,1400.00,0.00,45.78,0
2,85675,2007,Manipur,01/06/2007,1062010,10.0,RENT,Low,INDIVIDUAL,other,High,E,25000,1000,14.07,36 months,16.27,1231.38,1000.00,0.00,34.21,0
3,84918,2007,Andhra Pradesh,01/09/2007,1042008,10.0,MORTGAGE,Low,INDIVIDUAL,other,Low,A,65000,5000,7.43,36 months,0.28,5200.44,5000.00,0.00,155.38,0
4,84670,2007,Arunachal Pradesh,01/06/2007,1082009,10.0,MORTGAGE,High,INDIVIDUAL,other,Low,A,300000,5000,7.75,36 months,5.38,5565.65,5000.00,0.00,156.11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887374,68616825,2015,Nagaland,01/12/2015,1012016,2.0,RENT,Medium,INDIVIDUAL,debt_consolidation,Low,B,130000,20000,9.17,36 months,12.77,0.00,0.00,0.00,637.58,0
887375,68616851,2015,Telangana,01/12/2015,1012016,10.0,RENT,Low,INDIVIDUAL,credit_card,Low,A,52000,12500,7.49,36 months,6.83,0.00,0.00,0.00,388.78,0
887376,68616867,2015,Meghalaya,01/12/2015,1012016,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,High,C,80000,16000,14.85,60 months,19.59,247.39,181.39,0.00,379.39,0
887377,68616501,2015,Bihar,01/12/2015,1012016,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,High,D,100000,25000,18.49,60 months,5.58,0.00,0.00,0.00,641.52,0


In [75]:
%%time
# 2. cudf
cudf_data.query("cust_id > 83000")

CPU times: user 170 ms, sys: 4.35 ms, total: 174 ms
Wall time: 178 ms


,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
0,180675,2007,Andhra Pradesh,01/12/2007,1032009,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,Low,C,73000,25000,10.91,36 months,22.13,13650.38,8767.32,2207.65,817.41,1
1,85781,2007,Rajasthan,01/06/2007,1072010,0.5,RENT,Low,INDIVIDUAL,other,Low,C,40000,1400,10.91,36 months,8.61,1663.04,1400.00,0.00,45.78,0
2,85675,2007,Manipur,01/06/2007,1062010,10.0,RENT,Low,INDIVIDUAL,other,High,E,25000,1000,14.07,36 months,16.27,1231.38,1000.00,0.00,34.21,0
3,84918,2007,Andhra Pradesh,01/09/2007,1042008,10.0,MORTGAGE,Low,INDIVIDUAL,other,Low,A,65000,5000,7.43,36 months,0.28,5200.44,5000.00,0.00,155.38,0
4,84670,2007,Arunachal Pradesh,01/06/2007,1082009,10.0,MORTGAGE,High,INDIVIDUAL,other,Low,A,300000,5000,7.75,36 months,5.38,5565.65,5000.00,0.00,156.11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887374,68616825,2015,Nagaland,01/12/2015,1012016,2.0,RENT,Medium,INDIVIDUAL,debt_consolidation,Low,B,130000,20000,9.17,36 months,12.77,0.00,0.00,0.00,637.58,0
887375,68616851,2015,Telangana,01/12/2015,1012016,10.0,RENT,Low,INDIVIDUAL,credit_card,Low,A,52000,12500,7.49,36 months,6.83,0.00,0.00,0.00,388.78,0
887376,68616867,2015,Meghalaya,01/12/2015,1012016,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,High,C,80000,16000,14.85,60 months,19.59,247.39,181.39,0.00,379.39,0
887377,68616501,2015,Bihar,01/12/2015,1012016,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,High,D,100000,25000,18.49,60 months,5.58,0.00,0.00,0.00,641.52,0


In [77]:
%%time
# 3. daskcudf
daskcudf_data.query("cust_id > 83000").compute()

CPU times: user 52.2 ms, sys: 22.3 ms, total: 74.5 ms
Wall time: 75.7 ms


,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
0,180675,2007,Andhra Pradesh,01/12/2007,1032009,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,Low,C,73000,25000,10.91,36 months,22.13,13650.38,8767.32,2207.65,817.41,1
1,85781,2007,Rajasthan,01/06/2007,1072010,0.5,RENT,Low,INDIVIDUAL,other,Low,C,40000,1400,10.91,36 months,8.61,1663.04,1400.00,0.00,45.78,0
2,85675,2007,Manipur,01/06/2007,1062010,10.0,RENT,Low,INDIVIDUAL,other,High,E,25000,1000,14.07,36 months,16.27,1231.38,1000.00,0.00,34.21,0
3,84918,2007,Andhra Pradesh,01/09/2007,1042008,10.0,MORTGAGE,Low,INDIVIDUAL,other,Low,A,65000,5000,7.43,36 months,0.28,5200.44,5000.00,0.00,155.38,0
4,84670,2007,Arunachal Pradesh,01/06/2007,1082009,10.0,MORTGAGE,High,INDIVIDUAL,other,Low,A,300000,5000,7.75,36 months,5.38,5565.65,5000.00,0.00,156.11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887374,68616825,2015,Nagaland,01/12/2015,1012016,2.0,RENT,Medium,INDIVIDUAL,debt_consolidation,Low,B,130000,20000,9.17,36 months,12.77,0.00,0.00,0.00,637.58,0
887375,68616851,2015,Telangana,01/12/2015,1012016,10.0,RENT,Low,INDIVIDUAL,credit_card,Low,A,52000,12500,7.49,36 months,6.83,0.00,0.00,0.00,388.78,0
887376,68616867,2015,Meghalaya,01/12/2015,1012016,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,High,C,80000,16000,14.85,60 months,19.59,247.39,181.39,0.00,379.39,0
887377,68616501,2015,Bihar,01/12/2015,1012016,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,High,D,100000,25000,18.49,60 months,5.58,0.00,0.00,0.00,641.52,0


### **13.`isin` Method**

| Framework  | Pandas | cudf | daskcudf |
| ---- | ---- | ---- | ---- |
| Time | 49.9ms| 44.2ms| 67.5ms |


In [79]:
%%time
# 1. Pandas
data[data.state.isin(["Bihar", "Odisha"])]

CPU times: user 50.1 ms, sys: 0 ns, total: 50.1 ms
Wall time: 49.9 ms


,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
37,70686,2007,Bihar,01/06/2007,1062010,10.0,MORTGAGE,Low,INDIVIDUAL,other,Low,A,70000,5000,7.75,36 months,8.81,5619.72,5000.0,0.0,156.11,0
39,96844,2007,Bihar,01/07/2007,1072010,7.0,MORTGAGE,Low,INDIVIDUAL,credit_card,Low,A,74000,5300,8.38,36 months,14.37,6012.16,5300.0,0.0,167.02,0
48,92676,2007,Bihar,01/07/2007,1072010,0.5,MORTGAGE,Medium,INDIVIDUAL,home_improvement,Low,A,180000,5000,8.07,36 months,5.55,5645.90,5000.0,0.0,156.84,0
52,88854,2007,Bihar,01/08/2007,1032008,4.0,RENT,Medium,INDIVIDUAL,house,Low,A,200000,5000,7.43,36 months,0.28,5174.18,5000.0,0.0,155.38,0
63,72998,2007,Bihar,01/06/2007,1062010,0.5,RENT,Low,INDIVIDUAL,other,Low,B,12000,1000,9.64,36 months,10.00,1155.53,1000.0,0.0,32.11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887327,68615046,2015,Bihar,01/12/2015,1012016,7.0,RENT,Low,INDIVIDUAL,debt_consolidation,High,D,76000,16800,17.97,60 months,33.66,0.00,0.0,0.0,426.34,0
887348,68616039,2015,Bihar,01/12/2015,1012016,3.0,MORTGAGE,Medium,INDIVIDUAL,credit_card,Low,B,110000,12000,9.17,36 months,24.61,0.00,0.0,0.0,382.55,0
887355,68615444,2015,Bihar,01/12/2015,1012016,4.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,Low,A,50000,12000,5.32,36 months,15.25,0.00,0.0,0.0,361.38,0
887377,68616501,2015,Bihar,01/12/2015,1012016,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,High,D,100000,25000,18.49,60 months,5.58,0.00,0.0,0.0,641.52,0


In [81]:
%%time
# 2. cudf
cudf_data[cudf_data.state.isin(["Bihar", "Odisha"])]

CPU times: user 31.8 ms, sys: 10.9 ms, total: 42.7 ms
Wall time: 44.2 ms


,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
37,70686,2007,Bihar,01/06/2007,1062010,10.0,MORTGAGE,Low,INDIVIDUAL,other,Low,A,70000,5000,7.75,36 months,8.81,5619.72,5000.0,0.0,156.11,0
39,96844,2007,Bihar,01/07/2007,1072010,7.0,MORTGAGE,Low,INDIVIDUAL,credit_card,Low,A,74000,5300,8.38,36 months,14.37,6012.16,5300.0,0.0,167.02,0
48,92676,2007,Bihar,01/07/2007,1072010,0.5,MORTGAGE,Medium,INDIVIDUAL,home_improvement,Low,A,180000,5000,8.07,36 months,5.55,5645.90,5000.0,0.0,156.84,0
52,88854,2007,Bihar,01/08/2007,1032008,4.0,RENT,Medium,INDIVIDUAL,house,Low,A,200000,5000,7.43,36 months,0.28,5174.18,5000.0,0.0,155.38,0
63,72998,2007,Bihar,01/06/2007,1062010,0.5,RENT,Low,INDIVIDUAL,other,Low,B,12000,1000,9.64,36 months,10.00,1155.53,1000.0,0.0,32.11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887327,68615046,2015,Bihar,01/12/2015,1012016,7.0,RENT,Low,INDIVIDUAL,debt_consolidation,High,D,76000,16800,17.97,60 months,33.66,0.00,0.0,0.0,426.34,0
887348,68616039,2015,Bihar,01/12/2015,1012016,3.0,MORTGAGE,Medium,INDIVIDUAL,credit_card,Low,B,110000,12000,9.17,36 months,24.61,0.00,0.0,0.0,382.55,0
887355,68615444,2015,Bihar,01/12/2015,1012016,4.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,Low,A,50000,12000,5.32,36 months,15.25,0.00,0.0,0.0,361.38,0
887377,68616501,2015,Bihar,01/12/2015,1012016,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,High,D,100000,25000,18.49,60 months,5.58,0.00,0.0,0.0,641.52,0


In [83]:
%%time
# 3. daskcudf
daskcudf_data[daskcudf_data.state.isin(["Bihar", "Odisha"])].compute()

CPU times: user 55.6 ms, sys: 11.1 ms, total: 66.8 ms
Wall time: 67.5 ms


,cust_id,year,state,date_issued,date_final,emp_duration,own_type,income_type,app_type,loan_purpose,interest_payments,grade,annual_pay,loan_amount,interest_rate,loan_duration,dti,total_pymnt,total_rec_prncp,recoveries,installment,is_default
37,70686,2007,Bihar,01/06/2007,1062010,10.0,MORTGAGE,Low,INDIVIDUAL,other,Low,A,70000,5000,7.75,36 months,8.81,5619.72,5000.0,0.0,156.11,0
39,96844,2007,Bihar,01/07/2007,1072010,7.0,MORTGAGE,Low,INDIVIDUAL,credit_card,Low,A,74000,5300,8.38,36 months,14.37,6012.16,5300.0,0.0,167.02,0
48,92676,2007,Bihar,01/07/2007,1072010,0.5,MORTGAGE,Medium,INDIVIDUAL,home_improvement,Low,A,180000,5000,8.07,36 months,5.55,5645.90,5000.0,0.0,156.84,0
52,88854,2007,Bihar,01/08/2007,1032008,4.0,RENT,Medium,INDIVIDUAL,house,Low,A,200000,5000,7.43,36 months,0.28,5174.18,5000.0,0.0,155.38,0
63,72998,2007,Bihar,01/06/2007,1062010,0.5,RENT,Low,INDIVIDUAL,other,Low,B,12000,1000,9.64,36 months,10.00,1155.53,1000.0,0.0,32.11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887327,68615046,2015,Bihar,01/12/2015,1012016,7.0,RENT,Low,INDIVIDUAL,debt_consolidation,High,D,76000,16800,17.97,60 months,33.66,0.00,0.0,0.0,426.34,0
887348,68616039,2015,Bihar,01/12/2015,1012016,3.0,MORTGAGE,Medium,INDIVIDUAL,credit_card,Low,B,110000,12000,9.17,36 months,24.61,0.00,0.0,0.0,382.55,0
887355,68615444,2015,Bihar,01/12/2015,1012016,4.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,Low,A,50000,12000,5.32,36 months,15.25,0.00,0.0,0.0,361.38,0
887377,68616501,2015,Bihar,01/12/2015,1012016,10.0,MORTGAGE,Low,INDIVIDUAL,debt_consolidation,High,D,100000,25000,18.49,60 months,5.58,0.00,0.0,0.0,641.52,0


### **14.MultiIndex**

| Framework  | Pandas | cudf |
| ---- | ---- | ---- | 
| Time | 5.9ms| 12.9ms|

In [97]:
%%time
# 1. Pandas
arrays = [['a', 'a', 'b', 'b'], [1, 2, 3, 4]]
tuples = list(zip(*arrays))
idx = pd.MultiIndex.from_tuples(tuples)
display(idx)

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 3),
            ('b', 4)],
           )

CPU times: user 5.28 ms, sys: 27 µs, total: 5.31 ms
Wall time: 5.9 ms


In [98]:
%%time
# 2. cudf
arrays = [['a', 'a', 'b', 'b'], [1, 2, 3, 4]]
tuples = list(zip(*arrays))
idx = cudf.MultiIndex.from_tuples(tuples)
display(idx)

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 3),
            ('b', 4)],
           )

CPU times: user 13.3 ms, sys: 6 µs, total: 13.3 ms
Wall time: 12.9 ms
